<center>МИНИСТЕРСТВО НАУКИ И ВЫСШЕГО ОБРАЗОВАНИЯ РОССИЙСКОЙ ФЕДЕРАЦИИ </center>
<center>ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ОБРАЗОВАТЕЛЬНОЕ УЧРЕЖДЕНИЕ ВЫСШЕГО ОБРАЗОВАНИЯ </center>
<center>«НОВОСИБИРСКИЙ ГОСУДАРСТВЕННЫЙ ТЕХНИЧЕСКИЙ УНИВЕРСИТЕТ»</center>
<center>Кафедра Вычислительной техники </center>
<br>
<center> <b> <font size="5">  ОТЧЁТ </font>  </b>  </center>   
<center><font size="3">по лабораторной работе №1</font></center>
<center><font size="3">по дисциплине: «Системы искусственного интеллекта и машинное обучение» </font></center>
<br>

## Цель работы

Получить практические навыки работы по решению задачи первичного анализа «сырых» данных, подготовки массива к обработке, выявлению и описанию зависимостей в исходных данных с использованием языка программирования Python.

## Основная часть
---

### Первичный анализ датасета

**Предметная область** - доход и размеры домашних хозяйств Калифорнии по данным до 1990г.
Признаки в таблице:
1) Median House Value: Медианная цена дома в квартале[$]
2) Median Income: Медианный доход на семью в квартале[10тыс.$] (TARGET)
3) Median Age: Медианный возраст дома в квартале; меньше = новее [лет]
4) Total Rooms: Общее количество комнат в квартале
5) Total Bedrooms: Общее количество спален в квартале
6) Population: Общее количество людей, проживающих в квартале
7) Households: Общее количество домохозяйств, групп людей, проживающих в жилом помещении, в квартале
8) Latitude: показатель того, насколько далеко на север находится дом; чем выше значение, тем дальше на север [°]
9) Longitude: показатель того, насколько далеко на запад находится дом; чем выше значение, тем дальше на запад [°]
10) Distance to coast: расстояние до ближайшей точки побережья [м]
11) Distance to Los Angeles: расстояние до центра Лос-Анджелеса [м]
12) Distance to San Diego: расстояние до центра Сан-Диего [м]
13) Distance to San Jose: расстояние до центра Сан-Хосе [м]
14) Distance to San Francisco: расстояние до центра Сан-Франциско [м]
15) Distance to the ocean (Расстояние до океана)

Структура csv-файла:
- Ячейки разделены ";"
- Дробная часть помечена "."
- Нечисловой признак - ocean_proximity
- Дат не представлено
- Null помечены "?$?" (единственная запись)

In [ ]:
import pandas as pd
dataset_file = "datasets/California_Houses_15_col_dot.csv"
dataset = pd.read_csv(
    dataset_file,
    sep=';',
    decimal='.',
    header=0,
    na_values=['?$?']
)

dataset.head()

Выясним размеры датасета

In [ ]:
num_rows, num_cols = dataset.shape
print(f"Размеры набора данных: {num_rows} строк и {num_cols} столбцов\n")
dataset.info()

Посчитаем пропуски в записях

In [ ]:
for col in dataset.columns:
    print(f"{col}: {dataset[col].isna().sum()} пропусков")

Признак Tot_Rooms имеет тип float - следствие наличия в выборке NaN

In [ ]:
dataset.loc[dataset["Tot_Rooms"] % 1 != 0]

Признак *Median_Income* является nan лишь в двух записях 

In [ ]:
dataset[dataset.Median_Income.isna()]

Признак *ocean_proximity* - единственный нечисловой, представлен 5 уникальными строками

In [ ]:
if (dataset.dtypes == 'object').sum() >0:
    print(dataset.describe(include=['object', 'bool']).T )
dataset.ocean_proximity.value_counts()

В то же время он сильно связан с числовым признаком Distance_to_coast (за исключением случаев ISLAND, коих лишь 5)

In [ ]:
dataset.groupby("ocean_proximity")["Distance_to_coast"].describe()


Имеется дубликат признака Median_House_Value

In [ ]:
tmp = dataset.loc[dataset["Median_House_Value"] != dataset["median_house_value"]]
if tmp.empty:
    print("Median_House_Value полностью дублирует признак median_house_value")
else:
    print("Median_House_Value отличается от признака median_house_value")
    tmp

Уберем признак median_house_value из выборки

In [ ]:
dataset = dataset.drop(columns=["median_house_value"])

Во всем датасете на 20к записей лишь 3 пропуска -> исключим их по строкам

In [ ]:
print("Размер исходного DataFrame", dataset.shape)
print("Размер при исключении по строкам",  dataset.dropna(axis='index', how='any').shape)
dataset = dataset.dropna(axis='index', how='any')
if dataset.any().isna().sum() != 0:
    print("В данных остались пропуски")
else:
    print("Пропусков в данных не осталось")


Признак *ocean_proximity* - категориальный и не может быть однозначно перенесен на числовую ось. Используем прием One-Hot encoding

In [ ]:
obj_col_name    = "ocean_proximity"
prefix_col_name = "OcPrx"

dataset_dummies = pd.get_dummies(
    data   = dataset[obj_col_name],
    prefix = prefix_col_name,
    dtype  = int,
)

dataset_dummies.head()

Избавимся от нечислового *ocean_proximity*, заменив на выбранную кодировку

In [ ]:
dataset.drop(columns=[obj_col_name], inplace=True)
dataset = pd.concat([dataset, dataset_dummies], axis='columns')
dataset.head()

---

### Выбор целевого значения и признаков для анализа данных

Целевой признак - Median Income: Медианный доход на семью в квартале[10тыс.$].

Построим матрицу корреляции (коэфф. корреляции Пирсона) для возможных комбинай пар.

In [ ]:
corr_coeffs = dataset.corr(method='pearson')
corr_coeffs.style.background_gradient(cmap='coolwarm', axis=None)

In [ ]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

mask = np.zeros_like(corr_coeffs, dtype=bool)
mask[np.triu_indices_from(mask)] = True

fig, ax = plt.subplots(figsize=(12,12))
sns.set_theme(font_scale=0.8)

sns.heatmap(
            data=corr_coeffs,
            vmin=-1, vmax=1, center=0,
            annot=True,
            cmap = 'RdBu_r',
            mask=mask,
            square=True,
            linewidths=1.0,
            cbar_kws={"shrink": .7}
           )
plt.title(u'Матрица корреляции признаков');

In [ ]:
corr_coeffs["Median_Income"].abs().sort_values(ascending=False)

В качестве независимых переменных выберем признаки с высоким абс. значением коэфф. корреляции, но при этом как можно более не связанные между собой. Кандидаты:
- *Median_House_Value* - Медианная цена дома в квартале [$]
- *Distance_to_coast*- расстояние до ближайшей точки побережья [м] (или *OcPrx_INLAND*)
- *Tot_Rooms* - Общее количество комнат в квартале
- *Median_Age* - Медианный возраст дома в квартале; меньше = новее [лет]

In [ ]:
features = ['Median_House_Value', 'Distance_to_coast', 'Tot_Rooms',  'Median_Age']
target = ['Median_Income']

Построим попарные распределения признаков

In [ ]:
plt.figure(figsize=(7,7))
%config InlineBackend.figure_format = 'png'

sns.pairplot(
    dataset[features + target],
    kind="scatter",
    diag_kind = 'hist',
    plot_kws={'alpha':0.3}
);

Посмотрим на распределения самих значений - нужно будет отсеить выбросы

In [ ]:
for feature_name in (features+target):
    plt.figure(figsize=(16,2))
    sns.boxplot(data=dataset[[feature_name]], orient='h')

Зададим пределы

In [ ]:
threshold_map = {
    'Median_House_Value' : (0,480000),
    'Distance_to_coast' : (0,120000),
    'Tot_Rooms' : (0, 6000),
    'Median_Age' : (0, 60),
    'Median_Income' : (0, 8)
}
print(threshold_map)

И посмотрим, сколько значений выходит за эти границы

In [ ]:
for feature_name  in threshold_map:
    threshold_min =  threshold_map[feature_name][0]
    threshold_max =  threshold_map[feature_name][1]

    over_count = len(dataset[~((dataset[feature_name] >= threshold_min ) & (dataset[feature_name] <= threshold_max))])
    dcount = dataset[feature_name].count()

    print ('''{}. Принимаем пороговые значение равными {}.\nКол-во эл. вне пороговых - {} из {} значение(ий) ({:.1%})\n'''.format(
        feature_name,
        threshold_map[feature_name],
        over_count,
        dcount,
        over_count/ dcount,
    ))

Откинем все записи, где значения признаков выходят за рамки 

In [ ]:
dataset_filtered = dataset.copy()

for feature_name  in threshold_map:
    threshold_min =  threshold_map[feature_name][0]
    threshold_max =  threshold_map[feature_name][1]
    dataset_filtered = dataset_filtered[(dataset_filtered[feature_name] >= threshold_min ) & (dataset_filtered[feature_name] <= threshold_max)]
    print("Обработан ", feature_name )

print ("Отброшено {} строк. Осталось {} строк.".format(
    len(dataset)  - len(dataset_filtered),
    len(dataset_filtered))
)

Получим следующие распределения значений

In [ ]:
# dataset_filtered[features].hist() #Упрощенный вывод графиков

plt.figure(figsize=(16, 5))
plot_number = 0

for feature_name in (features+target):
    plot_number += 1
    plt.subplot(1, len(features+target), plot_number)
    plt.hist(dataset_filtered[feature_name])

    plt.title(feature_name)
    plt.xlabel(u'Значения')
    plt.ylabel(u'Частота')
    print (feature_name,
           dataset_filtered[feature_name].min(),
           dataset_filtered[feature_name].max())

Сохраним этот отфильтрованный датасет

In [ ]:
dataset_filtered.to_csv('datasets/dataset_prepared.csv', index=False)